# Opentrons Mass Balance Automation 

## Aniket Chitre

### 0. Import Packages

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
import serial
import csv 

### 1. Port for Mass Balance

Find the port number on Windows through Device Manager or on Mac through running this command (`ls /dev/tty.*`) on the Terminal

In [2]:
#port = '/dev/tty.usbserial-14130'
port = 'COM5'

### 2. Function to Read Mass from Balance

In [3]:
def get_mass(port):
    try:
        balance = serial.Serial(port, 9600, xonxoff=True) # check COM port and xonxoff (handshake) = True default for MT balance
        balance.write('S\r\n'.encode('utf-8'))
        time.sleep(1)
        result  = balance.read_until('\r\n'.encode('utf-8'))
        value = str(result[7:14].decode("utf-8"))
        #balance.close()
        if (value.strip() != '-------') and (value.strip() != ''):
            return (True,float(value))
        return (False,-1)
    except:
        return (False,-1)

### 3. Real-time Data

In [ ]:
x = [] #store time here 
mass = [] #store mass here
i = 0

while True:
    x.append(i*2)
    
    status, val = get_mass(port)
    if status == True:
        curr_val = val
    else:
        curr_val = mass[-1]
        
    print(curr_val)
    mass.append(curr_val)
    
    time.sleep(1)
    i += 1

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.001
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.003
-0.003
-0.004
-0.003
0.0
-0.001
-0.002
-0.001
1.0
0.999
0.999
0.999
1.998
1.998
1.997
1.998
1.998
2.996
2.995
2.995
3.994
3.993
3.992
3.991
4.991
4.99
4.988
4.988
4.99
5.99
5.987
6.812
6.812
6.81
6.81
6.81
7.631
7.631
7.63
7.631
7.631
8.629
8.63
8.63
8.629
9.624
9.627
9.63
9.628
9.628
10.623
10.626
10.625
10.624
10.628
11.621
11.624
11.626
11.624
11.626
12.625
12.623
12.623
12.625
13.621
13.621
13.621
13.622
14.436
14.439
14.44
14.438
14.44
15.254
15.257
15.26
15.259
15.26
16.257
16.256
16.255
16.256
17.252
17.252
17.25
17.25
18.246
18.244
18.244
18.244
19.239
19.239
19.238
19.238
20.234
20.234
20.232
20.233
21.229
21.228
21.227
21.869
21.869
21.868
22.508
22.507
22.506
22.506
23.502
23.502
23.5
23.5
24.495
24.494
24.493
24.493
25.488
25.488
26.481
26.481
26.479
26.479
27.476
27.474
27.473
27.473
28.468

57.244
57.244
57.244
57.243
57.243
57.243
57.242
57.242
57.242
57.242
57.242
57.242
57.241
57.241
57.241
57.241
57.241
57.24
57.24
57.239
57.239
57.239
57.238
57.238
57.238
57.238
57.238
57.237
57.236
57.234
57.234
57.234
57.234
57.238
57.235
57.235
57.235
57.235
57.235
57.234
57.234
57.234
57.234
57.234
57.234
57.234
57.234
57.233
57.233
57.233
57.232
57.232
57.232
57.231
57.23
57.23
57.23
57.23
57.23
57.23
57.23
57.23
57.23
57.23
57.228
57.229
57.229
57.229
57.229
57.229
57.228
57.229
57.228
57.228
57.227
57.227
57.226
57.226


In [ ]:
results = {'Time': x, 'Mass': mass}

### 4. Plotting and Exporting Data

In [ ]:
mass_df = pd.DataFrame(results)
mass_df

In [ ]:
sns.lineplot(x='Time', y='Mass', data=mass_df);

In [ ]:
mass_df.to_csv('mass_data/MassProfile_310123_S103-108.csv')